In [2]:
import pandas as pd

# Load the CSV file
file_path = 'all-the-news-2-1.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe
print(df.head())


                  date  year  month  day       author  \
0  2016-12-09 18:31:00  2016   12.0    9  Lee Drutman   
1  2016-10-07 21:26:46  2016   10.0    7  Scott Davis   
2  2018-01-26 00:00:00  2018    1.0   26          NaN   
3  2019-06-27 00:00:00  2019    6.0   27          NaN   
4  2016-01-27 00:00:00  2016    1.0   27          NaN   

                                               title  \
0  We should take concerns about the health of li...   
1  Colts GM Ryan Grigson says Andrew Luck's contr...   
2       Trump denies report he ordered Mueller fired   
3  France's Sarkozy reveals his 'Passions' but in...   
4  Paris Hilton: Woman In Black For Uncle Monty's...   

                                             article  \
0  This post is part of Polyarchy, an independent...   
1   The Indianapolis Colts made Andrew Luck the h...   
2  DAVOS, Switzerland (Reuters) - U.S. President ...   
3  PARIS (Reuters) - Former French president Nico...   
4  Paris Hilton arrived at LAX Wednesday

In [3]:
# Select the relevant columns
df = df[['title', 'publication']]

# Display the first few rows of the selected columns
print(df.head())

# Check for any missing values
print(df.isnull().sum())


                                               title       publication
0  We should take concerns about the health of li...               Vox
1  Colts GM Ryan Grigson says Andrew Luck's contr...  Business Insider
2       Trump denies report he ordered Mueller fired           Reuters
3  France's Sarkozy reveals his 'Passions' but in...           Reuters
4  Paris Hilton: Woman In Black For Uncle Monty's...               TMZ
title             37
publication    12577
dtype: int64


In [4]:
# Filter out less frequent publications (if needed)
# For example, keep only publications with more than 1000 articles
df = df[df['publication'].isin(df['publication'].value_counts()[df['publication'].value_counts() > 1000].index)]

# Display the remaining publications
print(df['publication'].value_counts())


Reuters               840094
The New York Times    252259
CNBC                  238096
The Hill              208411
People                136488
CNN                   127602
Refinery 29           111433
Vice                  101137
Mashable               94107
Business Insider       57953
The Verge              52424
TechCrunch             52095
TMZ                    49595
Axios                  47815
Vox                    47272
Politico               46377
Washington Post        40882
Buzzfeed News          32819
Gizmodo                27228
Economist              26227
Wired                  20243
Fox News               20144
Vice News              15539
Hyperallergic          13551
New Republic           11809
New Yorker              4701
Name: publication, dtype: int64


In [5]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['title'], df['publication'], test_size=0.2, random_state=42)


In [6]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['title'], df['publication'], test_size=0.2, random_state=42)


In [8]:
# Remove rows with missing titles
df = df.dropna(subset=['title'])

# Now, split the data into training and testing sets again
X_train, X_test, y_train, y_test = train_test_split(df['title'], df['publication'], test_size=0.2, random_state=42)

# Proceed with TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [9]:
 # Fill missing titles with an empty string
df['title'] = df['title'].fillna('')

# Now, split the data into training and testing sets again
X_train, X_test, y_train, y_test = train_test_split(df['title'], df['publication'], test_size=0.2, random_state=42)

# Proceed with TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix


In [11]:
# Train a Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)


MultinomialNB()

In [12]:
# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Print classification report
print(classification_report(y_test, y_pred))

# Print confusion matrix
print(confusion_matrix(y_test, y_pred))


                    precision    recall  f1-score   support

             Axios       0.44      0.01      0.01      9618
  Business Insider       0.47      0.06      0.11     11476
     Buzzfeed News       0.40      0.02      0.04      6441
              CNBC       0.50      0.28      0.36     47635
               CNN       0.27      0.10      0.15     25624
         Economist       0.91      0.13      0.23      5230
          Fox News       0.33      0.01      0.01      4025
           Gizmodo       0.31      0.03      0.05      5389
     Hyperallergic       0.62      0.09      0.16      2642
          Mashable       0.34      0.27      0.30     18885
      New Republic       0.50      0.00      0.00      2383
        New Yorker       1.00      0.07      0.13       948
            People       0.46      0.62      0.53     27327
          Politico       0.44      0.03      0.05      9320
       Refinery 29       0.58      0.45      0.50     22239
           Reuters       0.66      0.91

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

# Load the CSV file
file_path = 'all-the-news-2-1.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Remove rows with missing titles or dates
df = df.dropna(subset=['title', 'date'])

# Select relevant columns and sort by date
df = df[['title', 'publication', 'date']]
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')

# Split the data into training and testing sets based on time
split_date = '2017-01-01'  # Define your split date
train_df = df[df['date'] < split_date]
test_df = df[df['date'] >= split_date]

# Separate the features and labels
X_train = train_df['title']
y_train = train_df['publication']
X_test = test_df['title']
y_test = test_df['publication']

# Convert the text data to TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Ensure that labels are strings and inspect them
y_test = y_test.astype(str)
y_pred = [str(label) for label in y_pred]

# Print a sample of y_test and y_pred to debug
print("Sample of y_test:")
print(y_test.head())
print("Sample of y_pred:")
print(y_pred[:10])

# Print classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Sample of y_test:
1562316     Reuters
891114          CNN
1695417    The Hill
1424086        CNBC
1702704    The Hill
Name: publication, dtype: object
Sample of y_pred:
['Reuters', 'The Hill', 'The Hill', 'Reuters', 'The Hill', 'Reuters', 'People', 'Reuters', 'TMZ', 'The New York Times']
Classification Report:


/Users/anishchintamaneni/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anishchintamaneni/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anishchintamaneni/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

                    precision    recall  f1-score   support

             Axios       0.00      0.00      0.00     47808
  Business Insider       0.00      0.00      0.00     54710
     Buzzfeed News       0.27      0.02      0.03     24360
              CNBC       0.50      0.20      0.28    201532
               CNN       0.26      0.04      0.08    104054
         Economist       0.92      0.14      0.24     19408
          Fox News       0.00      0.00      0.00     20088
           Gizmodo       0.22      0.04      0.07     17473
     Hyperallergic       0.84      0.03      0.06     10188
          Mashable       0.21      0.31      0.25     60985
      New Republic       0.27      0.00      0.00      6910
        New Yorker       1.00      0.03      0.06      3468
            People       0.46      0.58      0.51    103783
          Politico       0.16      0.02      0.03     34143
       Refinery 29       0.65      0.29      0.41     88168
           Reuters       0.64      0.89

In [4]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix

# Load the CSV file
file_path = 'all-the-news-2-1.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Remove rows with missing titles or dates
df = df.dropna(subset=['title', 'publication'])

# Ensure that 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Handle invalid parsing

# Drop rows where 'date' is NaT after conversion
df = df.dropna(subset=['date'])

# Select relevant columns and sort by date
df = df[['title', 'publication', 'date']]
df = df.sort_values(by='date')

# Prepare the pipeline
pipeline = make_pipeline(TfidfVectorizer(max_features=5000), MultinomialNB())

# Initialize K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform K-Fold Cross-Validation
for fold, (train_index, test_index) in enumerate(kf.split(df)):
    X_train, X_test = df['title'].iloc[train_index], df['title'].iloc[test_index]
    y_train, y_test = df['publication'].iloc[train_index], df['publication'].iloc[test_index]
    
    # Convert labels to strings to avoid type issues
    y_train = y_train.astype(str)
    y_test = y_test.astype(str)
    
    # Train and evaluate the model
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    print(f"Fold {fold+1}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\n")


Fold 1
Classification Report:
                    precision    recall  f1-score   support

             Axios       0.37      0.01      0.01      9424
  Business Insider       0.48      0.06      0.11     11550
     Buzzfeed News       0.54      0.02      0.04      6674
              CNBC       0.50      0.28      0.36     47645
               CNN       0.26      0.10      0.14     25570
         Economist       0.93      0.13      0.23      5347
          Fox News       0.39      0.01      0.02      4052
           Gizmodo       0.32      0.03      0.05      5451
     Hyperallergic       0.68      0.09      0.16      2737
          Mashable       0.34      0.27      0.30     18805
      New Republic       0.45      0.00      0.00      2412
        New Yorker       0.99      0.07      0.14       956
            People       0.46      0.62      0.53     27376
          Politico       0.46      0.03      0.05      9118
       Refinery 29       0.58      0.45      0.51     22382
         

[[    47     12      1    816    301      5      0     10      0    152
       0      0    121     18     49   3758     18    195   2453   1334
     113    152      0     23      8     14]
 [     2    727     16   1455    282      4      0     20      0    680
       0      0    987      4    669   2881    125    184    772   1923
     513    517      0     28     16      7]
 [     5     22    150    248    576      0      1      9      3    413
       0      0    672      8     87    723     47     20   1040   1690
      76    615      0     28      8      1]
 [    15    222      3  13341    393     17      1     13      3    648
       0      0    394     30    238  25445     95    314   2571   2795
     532    486      0     63     12      9]
 [     4     56     28   1036   2607      3     14     36      1    574
       4      0   2052     78    374   6118    163     43   5143   6237
     163    626      1     65     34      3]
 [     1      6      0    309     54    699      0     

Fold 4
Classification Report:
                    precision    recall  f1-score   support

             Axios       0.39      0.01      0.01      9508
  Business Insider       0.49      0.06      0.11     11532
     Buzzfeed News       0.37      0.02      0.03      6610
              CNBC       0.50      0.28      0.36     47382
               CNN       0.27      0.10      0.15     25621
         Economist       0.91      0.13      0.23      5190
          Fox News       0.25      0.00      0.01      4100
           Gizmodo       0.30      0.03      0.05      5459
     Hyperallergic       0.61      0.09      0.16      2640
          Mashable       0.34      0.27      0.30     18871
      New Republic       0.33      0.00      0.00      2353
        New Yorker       0.98      0.06      0.12       912
            People       0.46      0.62      0.53     27359
          Politico       0.44      0.03      0.05      9368
       Refinery 29       0.57      0.44      0.50     22413
         

[[    43     11      3    861    310      5      3     19      0    132
       0      0    136     18     63   3714     17    180   2555   1304
     137    128      0     25      6      8]
 [     6    687      8   1395    282      6      0     14      1    719
       0      0    989      3    621   2792    148    201    784   1886
     458    537      0     17     10      7]
 [     2     31    143    252    588      2      6      8      0    451
       0      0    724     14     66    689     48     37    995   1789
      73    620      1     42      7      1]
 [    18    208      7  13323    426      7      1     16      0    670
       0      0    415     29    259  25390    104    372   2547   2860
     530    475      0     53     17      5]
 [     8     62     16   1031   2514      3     16     39      1    580
       3      0   2020     79    405   6071    159     35   5193   6516
     151    584      0     69     18      4]
 [     0      5      0    294     58    700      0     

In [ ]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix

# Load the CSV file
file_path = 'all-the-news-2-1.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Remove rows with missing titles or publications
df = df.dropna(subset=['title', 'publication'])

# Ensure that 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Handle invalid parsing

# Drop rows where 'date' is NaT after conversion
df = df.dropna(subset=['date'])

# Select relevant columns and sort by date
df = df[['title', 'publication', 'date']]
df = df.sort_values(by='date')

# Prepare the pipeline
pipeline = make_pipeline(TfidfVectorizer(max_features=5000), MultinomialNB())

# Initialize Leave-One-Out Cross-Validation
loo = LeaveOneOut()

# Initialize lists to store results
reports = []
confusion_matrices = []

# Perform Leave-One-Out Cross-Validation
for train_index, test_index in loo.split(df):
    X_train, X_test = df['title'].iloc[train_index], df['title'].iloc[test_index]
    y_train, y_test = df['publication'].iloc[train_index], df['publication'].iloc[test_index]
    
    # Convert labels to strings to avoid type issues
    y_train = y_train.astype(str)
    y_test = y_test.astype(str)
    
    # Train and evaluate the model
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    # Store the results for this fold
    reports.append(classification_report(y_test, y_pred, output_dict=True, zero_division=0))
    confusion_matrices.append(confusion_matrix(y_test, y_pred))

# Aggregate and print results
print("Leave-One-Out Cross-Validation Results:")
for i, (report, cm) in enumerate(zip(reports, confusion_matrices), start=1):
    print(f"Iteration {i}")
    print("Classification Report:")
    print(report)
    print("Confusion Matrix:")
    print(cm)
    print("\n")
